# Improved Tree Algorithm

we will show the complexity calculation of each of the relevant functions and sum up the complexity for the algorithm as a whole.

In [1]:
import numpy as np
import copy
import math
import itertools
from functools import cache

### Optimisation route - using a binary char tree

We implemented a ranked char tree to save going over each entire sequence when computing  $\overline{U^{a,i}}$, and to make get_ordering more efficient.

**each node contains the following ranks:**

* height - height of the subtree.
* size - number of leaves in the subtree.
* contains_1_vector - is the $\overline{1}$ vector contained in the subtree.
* contains_0_vector - is the $\overline{0}$ vector contained in the subtree.

In [2]:
class SequenceTreeNode:
    def __init__(self):
        self.children = np.array([None, None])
        self.char = -1
        self.height = 0
        self.size = 0
        self.contains_0_vector = False
        self.contains_1_vector = False

    def has_sequence(self, x):
        node = self
        for i,c in enumerate(x):
            if node.children[c] is None:
                return False
            node = node.children[c]
        return True

    def insert_sequence(self, x):
        if self.has_sequence(x):
            return

        self.__insert_sequence_internal(x)

    def __insert_sequence_internal(self, x):
        self.size += 1
        if x.size == 0:
            return

        self.height = x.size

        if np.all(x==0):
            self.contains_0_vector = True   
        elif np.all(x==1):
            self.contains_1_vector = True 

        letter = x[0]
        if self.children[letter] is None:
            self.children[letter] = SequenceTreeNode()

        self.children[letter].char = letter
        self.children[letter].__insert_sequence_internal(x[1:])

    def print_tree(self):
        self.__print_tree(0)

    def __print_tree(self, depth):
        print(f'({depth}) - char {self.char} has {self.size} sequences.\n\t\
            contains the 0 vector: {self.contains_0_vector}\n\t\
            contains the 1 vector: {self.contains_1_vector}')
        for child in self.children:
            if child is not None:
               child.__print_tree(depth+1)

    def print_all_sequences(self):
        self.__print_all_sequences(np.array([], dtype=np.ubyte))

    
    def __print_all_sequences(self, path):
        if np.all(self.children == None):
            print(path)
        for child in self.children:
            if child is None:
                continue
            child.__print_all_sequences(np.append(path, child.char))

    def decrease_size_by(self, k):
        if k <= 0:
            return

        self.size -= k

        if self.children[0] is not None and self.children[0].size <= k:
            self.children[1].decrease_size_by(k - self.children[0].size)
            self.children[0] = None
            return
        elif self.children[1] is not None and self.children[1].size <= k:
            self.children[0].decrease_size_by(k - self.children[1].size)
            self.children[1] = None
            return

        if self.children[0] is not None:
            self.children[0].decrease_size_by(k)
        else: 
            self.children[1].decrease_size_by(k)
        

    def decrease_size_to(self, N):
        self.decrease_size_by(self.size - N)

    def get_unique_path(self):
        if self.size != 1:
            raise Exception()

        path = []

        node = self
        while node.height > 0:
            if node.children[0] is not None:
                path.append(0)
                node = node.children[0]
            else:
                path.append(1)
                node = node.children[1]
        
        return np.array(path, dtype=int)

In [3]:
U = SequenceTreeNode()
U.insert_sequence(np.array([0, 1, 0, 1, 1, 0]))
print(U.get_unique_path())

[0 1 0 1 1 0]


**get_u_a_i:**

input - $U$ sequence tree of height $n$, a char in $F_{2}$, $i$ index $\leq n$.

output - $\overline{U^{a,i}}$.

Returns the correct subtree whose root is unde the path $p=(1-a, 1-a,\dots, 1-a, a)$ where $p$'s length is $i$. In total to get the subtree the complexity is $O\left(i\right)$.

In [4]:
def get_u_a_i(U, a, i):
    root = SequenceTreeNode()
    node = U
    for j in range(i-1):
        node = node.children[1-a]        
        if node is None:
            return root
    
    node = node.children[a]
    if node is None:
        return root

    root = copy.copy(node)
    root.char = -1
    
    return root

# Reconstruction from subsequences tree algorithm:

Reconstruct a sequence $x$ of size $n$ from $N_q^-(n,t)+1$ of $x's$ subsequences tree of size $n-t$.

**get_composition_vector:**

input - $U$ sequence tree.

output - $k$ composition vector.

uses the rank of the tree to calculate how many vectors start with $0$ or $1$ with complexity of $O\left(1\right)$.

In [5]:
def get_composition_vector(U):
    k = np.zeros_like(U.children)
    for i, child in enumerate(U.children):
        if child is None:
            k[i] = 0
        else:
            k[i] = child.size
    return k

**get_ordering:**

input - $U$ sequence tree.

output - $t$ ordered permutation, $c$ ordered composition.

uses get_composition_vector $O\left(1\right)$, argsort on the histogram to return the ordering permutation $O\left(q\log\left(q\right)\right)$, and returns the ordered composition $O\left(q\right)$.

$q=2$ is a constant so get ordering has the complexity of $O\left(1\right)$.

In [6]:
def get_ordering(U):
    k = get_composition_vector(U)
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

**get_maximal_deletion_ball_size:**

input - $n$ original sequence size, $t$ deletion ball radius.

output - maximal deletion ball size.

loops for $t+1$ iterations with counter i and in each computes ${n-t \choose i}$ since we know $t<<n$ this has the complexity $O\left(n\right)$.

in total the function has the complexity of $O\left(nt\right)$.

In [7]:
@cache
def get_maximal_deletion_ball_size(n, t, q=2):
    if n < t or t < 0:
        return 0
    if q == 1:
        return 1

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i) * get_maximal_deletion_ball_size(t, t-i, q-1)
    return size

**get_maximal_number_of_common_subsequences:**

input - $n$ original sequence size, $t$ deletion ball radius, $q$ alphabet size - constant.

output - maximal number of common subsequences.

calls get_maximal_deletion_ball_size $3$ times with $n\pm const$, $t\pm const$ which has a complexity of $O\left(3nt\right)=O\left(nt\right)$.

In [8]:
@cache
def get_maximal_number_of_common_subsequences(n, t):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t) - get_maximal_deletion_ball_size(n-1,t) + get_maximal_deletion_ball_size(n-2,t-1)

**get_subsequence_reconstruction_threshold:**

input - $n$ original sequence size, $t$ insertion ball radius, $order\_comp$ ordered composition (vec of size $q$), $q$ alphabet size - constant.

output - threshold on ordered composition using $\tau_{i}=N_{q}^{-}\left(n-i-1,\,t-i\right)$.

loops $q$ times and calls get_maximal_number_of_common_subsequences on each iteration $O\left(qnt\right)=O\left(nt\right)$.

In [9]:
def get_subsequence_reconstruction_threshold(n, t, order_comp):
    for i, w_i in enumerate(order_comp):
        tau_i = get_maximal_number_of_common_subsequences(n-i-1,t-i)
        if w_i > tau_i:
            return i

### reconstruct_x_from_subsequences_tree:

The following algorithm reconstructs a sequence $X \in F_2^n$ given $N_2^-(n,t)+1$ different subsequences of $x$ of size $n-t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $x$, this is a sequence tree of height $n-t$ and size of at least $N_2^-(n,t)+1$
* **verbose**       - control verbosity.


**Complexity:**

The algorithm loops at most $n$ times, in each iteration:

• call get_ordering, complexity $\leq O\left(1\right)$.

• call get_subsequence_reconstruction_threshold, complexity $\leq O\left(nt\right)$.

• concatenate, complexity $\leq O\left(n\right)$.

• call get_maximal_number_of_common_subsequences, complexity $\leq O\left(nt\right)$.

• call get_u_a_i (with i==1), complexity $\leq O\left(1\right)$.

getting us in total a complexity of $O\left(n^{2}t\right)$.

In [10]:
def reconstruct_x_from_subsequences_tree(n, U, verbose=False):
    t = n-U.height
    reconstruction = np.array([], dtype=int)
    N = get_maximal_number_of_common_subsequences(n,t)+1
    while t >= 1:
        order_perm, order_comp = get_ordering(U)
        if verbose:
            print(f'n: {n}, t: {t}, U: ({U.size}, {U.height}), N: {N}')
            print(f'perm: {order_perm}, comp: {order_comp}')
        j = get_subsequence_reconstruction_threshold(n, t, order_comp)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if verbose:
            print(j, reconstruction)

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t)+1
        U = get_u_a_i(U, order_perm[j], 1)
        U.decrease_size_to(N)
        
    if verbose:
        print(U.size, U.get_unique_path())

    return np.concatenate((reconstruction, U.get_unique_path()))

Using the optimized algorithm for q=2:

Use get_deletion_ball to construct the $t$ deletion ball of $x$, we will use $N_q^-(n,t)+1$ sequences from it to reconstruct $x$.

In [11]:
def get_deletion_ball_tree(x, t, N=-1):
    result = SequenceTreeNode()
    for c in itertools.combinations(x, x.size-t):
        result.insert_sequence(np.array(c))
        if result.size == N:
            break

    return result

In [12]:
n = 100
t = 3
q = 2

x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t)+1
D_x = get_deletion_ball_tree(x, t, N)

while D_x.size < N:
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball_tree(x, t, N)

In [13]:
subsequences = get_deletion_ball_tree(x, t, N)

In [14]:
%%timeit -r1 -n1

print(f'n = {n}, t = {t}, q = {q}, N = {N}')
reconstructed_x = reconstruct_x_from_subsequences_tree(n, subsequences)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

n = 100, t = 3, q = 2, N = 9315
Reconstructed [1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1
 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0
 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 0]
From [1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1
 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0
 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 0]
True
23.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Reconstruction from supersequences tree algorithm:

Reconstruct a sequence $x$ of size $n$ from $N_q^+(n,t)+1$ of $x's$ supersequences tree of size $n+t$.

**get_maximal_insertion_ball_size:**

input - $n$ original sequence size, $t$ insertion ball radius, $q$ alphabet size - constant.

output - maximal insertion ball size.

assuming $q=2$ the recursive call in the function will only take $O\left(1\right)$ complexity.

loops for $t+1$ iterations with counter $i$ and in each computes ${n+t \choose i}$ since we know $t<<n$ this has the complexity $O\left(n\right)$.

in total the function has the complexity of $O\left(nt\right)$.

In [15]:
@cache
def get_maximal_insertion_ball_size(n, t):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)
    return size

**get_maximal_number_of_common_supersequences:**

input - $n$ original sequence size, $t$ insertion ball radius.

loop $t$ times, on each loop call get_maximal_insertion_ball_size getting us an overall complexity of $O\left(nt^{2}\right)$.

In [16]:
@cache
def get_maximal_number_of_common_supersequences(n, t):
    if t <= 0:
        return 0
    result = 0
    for i in range(1, t+1):
        result += get_maximal_insertion_ball_size(n,t-i)
    return 2*result

**get_m_vector:**

input - $U$ sequence tree of height $n$ and size $N$, $t$ insertion ball radius, $a$ char in $F_{2}$.

output - $m\left(a\right)$ vector.

loop $t+1$ times, with counter $i$, on each loop call get_u_a_i($i+1$) with complexity of $O\left(i\right)$.

getting us a total complexity of $O\left(t^{2}\right)$.

In [17]:
def get_m_vector(U, t, a):
    m_vec = np.zeros((t+1,), dtype=np.uint)
    for i in range(t+1):
        u_a_i = get_u_a_i(U, a, i+1)
        m_vec[i] = u_a_i.size

    return m_vec

**get_first_x:**

input - $U$ supersequence tree of height $n$ and size $N$, $t$ insertion ball radius.

output - $x_{1}$ first letter of $x$, $m\left(x_{1}\right)$ the $m$ vector of $x_{1}$ as defined in the paper. 

calls get_m_vector 2 times $O\left(t^{2}\right)$, sums each the two $m\left(a\right)$ vectors (which are of size $t+1$) $O\left(t\right)$, calculates $U_{0,1}$ and $U_{1,0}$ using the sequence tree ranks with complexity of $O(1)$

in summary we get a complexity of $O\left(t^{2}\right)$.

In [18]:
def get_first_x(U, t):
    N = U.size
    candidates = []
    for i in range(2):
        m_v = get_m_vector(U, t, i)
        if m_v.sum() == N:
            candidates.append((i, m_v))

    if len(candidates) == 1:
        return candidates[0]

    U_0_1 = 0
    U_1_0 = 0

    if U.children[0] is not None:
        U_0_1 = U.children[0].size - (1 if U.contains_0_vector else 0)
    if U.children[1] is not None:
        U_1_0 = U.children[1].size - (1 if U.contains_1_vector else 0)

    # Since len(candidates)>1 then both 0 and 1 are candidates 
    # and because we added them by order candidates[0] represents 0
    if U_0_1 > U_1_0:
        return candidates[0] 
    
    return candidates[1]

**get_supersequence_reconstruction_threshold:**

input - $n$ length of sequence, $t$ insertion ball radius, $m\left(x_{1}\right)$ $m$ vector of $x_{1}$.

output - threshold for $m\left(x_{1}\right)$ with $\tau_{i}=N_{q}^{+}\left(n-1,t-i\right)$.

loop $t$ times, with counter $i$, calculate $\tau_{i}$ on each loop $O\left(ni^{2}\right)$.

getting us a total complexity $O\left(nt^{3}\right)$.

In [19]:
def get_supersequence_resonstruction_threshold(n, t, m_v):
    t = m_v.size - 1
    for i, w_i in enumerate(m_v):
        tau_i = get_maximal_number_of_common_supersequences(n-1,t-i)
        if w_i > tau_i:
            return i

### reconstruct_x_from_supersequences_tree:

The following algorithm reconstructs a sequence $x \in F_2^n$ given $N_2^+(n,t)+1$ different subsequences of $x$ of size $n+t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $x$, this is a subsequence tree of height $n+t$ and size of at least $N_q^+(n,t)+1$.
* **verbose**       - control verbosity.

**Complexity:**
The algorithm loops at most $n$ times:

• get_first_x, complexity $\leq O\left(t^{2}\right)$.

• get_supersequence_reconstruction_threshold, complexity $\leq O\left(nt^{3}\right)$.

• get_u_a_i, complexity $\leq O\left(t\right)$.

• get_maximal_number_of_common_supersequences, complexity $\leq O\left(nt^{2}\right)$.

• concatenations, complexity $\leq\left(n\right)$.

is summary we get a complexity of $O\left(n^{2}t^{3}\right)$.


In [20]:
def reconstruct_x_from_supersequences_tree(n, U, verbose=False):
    t = U.height-n
    N = get_maximal_number_of_common_supersequences(n,t)+1
    reconstruction = np.array([], dtype=int)
    while t >= 1:
        if verbose:
            print(f'n: {n}, t: {t}, U: ({U.size}, {U.height}), N: {N}')
        x_1, m_x_1 = get_first_x(U, t)
        reconstruction = np.append(reconstruction, x_1)
        if n == 1: 
            return reconstruction
        j = get_supersequence_resonstruction_threshold(n, t, m_x_1)
        if verbose:
            print(j, reconstruction)

        U = get_u_a_i(U, x_1, j+1)
        n = n-1
        t = t-j  
        N = get_maximal_number_of_common_supersequences(n,t)+1
        U.decrease_size_to(N)

    return np.concatenate((reconstruction, U.get_unique_path()))

Using the algorithm for q=2:

Use get_insertion_ball to construct the $t$ insertion ball of $x$, we will use $N_q^+(n,t)+1$ sequences from it to reconstruct $x$.

In [21]:
def get_insertion_ball_tree(x, t, N=-1):
    result = SequenceTreeNode()
    n = x.shape[0]
    for indices in itertools.combinations_with_replacement(range(n+1), t):
        for letters in itertools.product(range(q), repeat=t):
            seq = np.insert(x, indices, letters)
            result.insert_sequence(seq)

            if result.size == N:
                return result

    return result

In [22]:
n = 100
t = 3
q = 2

x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_supersequences(n,t)+1
I_x = get_insertion_ball_tree(x, t, N)
if I_x.size < N:
    raise Exception("Ball is too small, change parameters")

In [23]:
supersequences = get_insertion_ball_tree(x, t, N)

In [24]:
%%timeit -r1 -n1

print(f'n = {n}, t = {t}, q = {q}, N = {N}')
reconstructed_x = reconstruct_x_from_supersequences_tree(n, supersequences)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

n = 100, t = 3, q = 2, N = 10715
Reconstructed [1 0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0
 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0
 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0]
From [1 0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0
 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0
 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0]
True
58.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
